In [ ]:
import heapq
import torch
import pandas as pd
import numpy as np
import math
from utilities import get_test_instances_with_random_samples
from collaborative_models import neuralCollabFilter

emb_size = 128
hidden_layers = np.array([emb_size, 64, 32, 16])
output_size = 1

train_data = pd.read_csv("train-test/train_userPages.csv")
test_data = pd.read_csv("train-test/test_userPages.csv")
num_unique_users = len(train_data.user_id.unique())
num_unique_likes = len(train_data.like_id.unique())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pre = neuralCollabFilter(num_unique_users, num_unique_likes, emb_size, hidden_layers,output_size).to(device)

pre.load_state_dict(torch.load("trained-models/preTrained_NCF"))


pre.to(device)

In [ ]:
# %% model evaluation: hit rate and NDCG
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0


def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i + 2)
    return 0

def evaluate_model(model, df_val, top_K, random_samples, num_items):
    model.eval()
    avg_HR = np.zeros((len(df_val), top_K))
    avg_NDCG = np.zeros((len(df_val), top_K))

    for i in range(len(df_val)):
        test_user_input, test_item_input = get_test_instances_with_random_samples(df_val[i], random_samples, num_items,
                                                                                  device)
        y_hat = model(test_user_input, test_item_input)
        y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
        test_item_input = test_item_input.cpu().detach().numpy().reshape((-1,))
        map_item_score = {}
        for j in range(len(y_hat)):
            map_item_score[test_item_input[j]] = y_hat[j]
        for k in range(top_K):
            # Evaluate top rank list
            ranklist = heapq.nlargest(k, map_item_score, key=map_item_score.get)
            gtItem = test_item_input[0]
            avg_HR[i, k] = getHitRatio(ranklist, gtItem)
            avg_NDCG[i, k] = getNDCG(ranklist, gtItem)
    avg_HR = np.mean(avg_HR, axis=0)
    avg_NDCG = np.mean(avg_NDCG, axis=0)
    return avg_HR, avg_NDCG

random_samples = 100
top_K = 10

ht, ndcg = evaluate_model(pre, test_data.values, top_K, random_samples,
                          num_unique_likes)

In [ ]:
ht[-1]

In [ ]:
new = neuralCollabFilter(num_unique_users, 3952, emb_size, hidden_layers,output_size).to(device)

new.load_state_dict(torch.load("trained-models/new_pre"))

In [ ]:
ht2, ndcg2 = evaluate_model(new, test_data.values, top_K, random_samples,
                            3952)

In [ ]:
import matplotlib.pyplot as plt
ht2[-1]
